In [1]:
# Import libraries
import re
import pickle
import random
import numpy as np
import datapackage
import pandas as pd
import matplotlib.pyplot as plt

# Import dependencies3
from os import listdir
from os.path import isfile, join
from pipeline_utils import *
from datetime import datetime, timedelta

# Other configurations
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
data_path = "../local_path"                                         # DATA PATH
resource_path = "../mimic-iv-v2-0/resources/"  # PIPELINE PATH

In [2]:
typical_cols = ['subject_id', 'hadm_id', 'stay_id', 'charttime', 'itemid', 'value', 'valuenum', 'valueuom', 'linksto', 'level1']
folder_path = data_path + "/vitals_and_labs"
lab_files = os.listdir(folder_path)

In [ ]:
# num_rows = 20
# num_cols = 5

# fig, axes = plt.subplots(num_rows, num_cols, figsize = (20, 60))
# axes = axes.flatten()

# for idx, lab_type in enumerate(lab_files):
#     lab_df = pd.read_csv(folder_path + "/" + lab_type, low_memory = False)
    
#     if lab_type == "gcs.csv":
#         lab_df = lab_df[lab_df['gcs'] != 999999]
#         axes[idx].hist(lab_df['gcs'], bins = 50, color = 'dodgerblue', edgecolor = 'black')
#     elif lab_type == "urine_output.csv":
#         lab_df = lab_df[lab_df['urineoutput'] != 999999]
#         axes[idx].hist(lab_df['urineoutput'], bins = 50, color = 'dodgerblue', edgecolor = 'black')
#     else:
#         lab_df = lab_df[lab_df['valuenum'] != 999999]
#         axes[idx].hist(lab_df['valuenum'], bins = 50, color = 'dodgerblue', edgecolor = 'black')
    
#     axes[idx].set_title(lab_type)

# plt.tight_layout()
# plt.show()

In [ ]:
# def filter_by_mad(df, num_devs = 10):
#     median = df.median()
#     mad = (df - median).abs().median()
#     lower_bound = median - num_devs * mad
#     upper_bound = median + num_devs * mad
#     filtered_df = df.loc[(df >= lower_bound) & (df <= upper_bound)]
#     return filtered_df

# fig, axes = plt.subplots(num_rows, num_cols, figsize = (20, 60))
# axes = axes.flatten()

# for idx, lab_type in enumerate(lab_files):
#     lab_df = pd.read_csv(folder_path + "/" + lab_type, low_memory = False)
    
#     if lab_type == "gcs.csv":
#         lab_df = lab_df[lab_df['gcs'] != 999999]
#         lab_df = filter_by_mad(lab_df['gcs'])
#         axes[idx].hist(lab_df, bins = 40, color = 'dodgerblue', edgecolor = 'black')
#     elif lab_type == "urine_output.csv":
#         lab_df = lab_df[lab_df['urineoutput'] != 999999]
#         lab_df = filter_by_mad(lab_df['urineoutput'])
#         axes[idx].hist(lab_df, bins = 40, color = 'dodgerblue', edgecolor = 'black')
#     else:
#         lab_df = lab_df[lab_df['valuenum'] != 999999]
#         lab_df = filter_by_mad(lab_df['valuenum'])
#         axes[idx].hist(lab_df, bins = 40, color = 'dodgerblue', edgecolor = 'black')
    
#     axes[idx].set_title(lab_type)

# plt.tight_layout()
# plt.show()

In [ ]:
# def filter_by_quantiles(series):
#     lower_bound = series.quantile(0.01)
#     upper_bound = series.quantile(0.99)
#     return series[(series >= lower_bound) & (series <= upper_bound)]

# fig, axes = plt.subplots(num_rows, num_cols, figsize = (20, 60))
# axes = axes.flatten()

# for idx, lab_type in enumerate(lab_files):
#     lab_df = pd.read_csv(folder_path + "/" + lab_type, low_memory = False)
    
#     if lab_type == "gcs.csv":
#         lab_df = lab_df[lab_df['gcs'] != 999999]
#         lab_df = filter_by_quantiles(lab_df['gcs'])
#         axes[idx].hist(lab_df, bins = 40, color = 'dodgerblue', edgecolor = 'black')
#     elif lab_type == "urine_output.csv":
#         lab_df = lab_df[lab_df['urineoutput'] != 999999]
#         lab_df = filter_by_quantiles(lab_df['urineoutput'])
#         axes[idx].hist(lab_df, bins = 40, color = 'dodgerblue', edgecolor = 'black')
#     else:
#         lab_df = lab_df[lab_df['valuenum'] != 999999]
#         lab_df = filter_by_quantiles(lab_df['valuenum'])
#         axes[idx].hist(lab_df, bins = 40, color = 'dodgerblue', edgecolor = 'black')
    
#     axes[idx].set_title(lab_type)

# plt.tight_layout()
# plt.show()

In [4]:
static_df = load_static_df(data_path, resource_path)

In [5]:
selected_static_df = select_patient_cohort(static_df, min_age = 18, max_age = 90, min_duration = 24, max_duration = None,
                                           hosq_seq = None, icustay_seq = None, mort_hosp = None,
                                           mort_icu = None, readmission_30 = None)

selected_patients = selected_static_df[selected_static_df['icustay_seq'] != 1]['subject_id'].unique()
selected_static_df = selected_static_df[selected_static_df['subject_id'].isin(selected_patients)]
selected_static_df = selected_static_df[selected_static_df['icustay_seq'] == 1]
print(len(selected_static_df))

7041


In [9]:
variable_range_df = get_variable_ranges(resource_path)
union_df = aggr_dynamic_features(selected_features_and_levels = ['mean_blood_pressure(arterial)'],
                                 icu_ids = selected_static_df.index,
                                 data_path = data_path, 
                                #  use_table = True,
                                 variable_ranges = variable_range_df,
                                 exclusion_method = 'Outlier',
                                 alternative_method = 'MAD'
                                 )

labs_df = union_df.copy()
union_df = union_df[union_df['mean_blood_pressure'] <= 65]
conditional = union_df.groupby(['stay_id'])['mean_blood_pressure'].count() >= 7
true_stay_ids = [stay_id for stay_id, value in conditional.items() if value]
labs_df = labs_df.loc[true_stay_ids].reset_index()

labs_df.head()

Finished reading mean_blood_pressure.csv


,stay_id,charttime,mean_blood_pressure
0,30009597,2175-10-05 08:23:00,101.0
1,30009597,2175-10-05 09:00:00,71.0
2,30009597,2175-10-05 10:00:00,69.0
3,30009597,2175-10-05 11:00:00,80.0
4,30009597,2175-10-05 12:00:00,90.0


In [10]:
merged_df = pd.merge(labs_df, selected_static_df[['admittime']], on = 'stay_id', how = 'left')

# Convert charttime and admittime to datetime if they are not already
merged_df['charttime'] = pd.to_datetime(merged_df['charttime'])
merged_df['admittime'] = pd.to_datetime(merged_df['admittime'])

# Calculate the time difference between charttime and admittime
merged_df['time_diff_hours'] = (merged_df['charttime'] - merged_df['admittime']).dt.total_seconds() / 3600

# Filter rows where the time difference is within the initial 48 hours
filtered_df = merged_df[merged_df['time_diff_hours'] <= 48]
labs_df = filtered_df.drop(columns = ['admittime', 'time_diff_hours'])

labs_df.head(20)

,stay_id,charttime,mean_blood_pressure
53,30014984,2127-07-01 07:30:00,69.0
54,30014984,2127-07-01 08:00:00,72.0
55,30014984,2127-07-01 09:00:00,67.0
56,30014984,2127-07-01 10:00:00,71.0
57,30014984,2127-07-01 11:01:00,78.0
58,30014984,2127-07-01 12:00:00,71.0
59,30014984,2127-07-01 13:00:00,81.0
60,30014984,2127-07-01 14:00:00,63.0
61,30014984,2127-07-01 15:00:00,66.0
62,30014984,2127-07-01 16:00:00,69.0


In [11]:

df_Norepinephrine = pd.read_csv("../local_path/treatments/norepinephrine.csv")
df_Vasopressin =  pd.read_csv("../local_path/treatments/vasopressin.csv")
df_Phenylephrine =  pd.read_csv("../local_path/treatments/phenylephrine.csv")
df_Dopamine =  pd.read_csv("../local_path/treatments/dopamine.csv")
df_Epinephrine =  pd.read_csv("../local_path/treatments/epinephrine.csv")

df_crystalloid_bolus = pd.read_csv("../local_path/treatments/crystalloid_bolus.csv")
df_colloid_bolus = pd.read_csv("../local_path/treatments/colloid_bolus.csv")


In [12]:
df_colloid_bolus.head(5)
df_colloid_bolus = df_colloid_bolus.sort_values('charttime')
df_colloid_bolus.head(5)

df_crystalloid_bolus.head(5)
df_crystalloid_bolus = df_crystalloid_bolus.sort_values('charttime')
df_crystalloid_bolus.head(5)



,stay_id,charttime,crystalloid_bolus
10106,30588857,2110-01-11 13:00:00,500
10107,30588857,2110-01-11 14:15:00,500
10108,30588857,2110-01-11 15:53:00,500
10109,30588857,2110-01-11 17:16:00,500
10110,30588857,2110-01-12 01:50:00,500


In [13]:
from datetime import datetime, timedelta
def compare_timestamps(time1, time2):
    format = "%Y-%m-%d %H:%M:%S"
    datetime1 = datetime.strptime(time1, format)
    datetime2 = datetime.strptime(time2, format)
    if datetime1 < datetime2:
        return -1
    elif datetime1 > datetime2:
        return 1
    else:
        return 0
    



In [21]:
print(df_colloid_bolus.head())
df_colloid_bolus.reset_index(drop=True, inplace=True)
print(df_colloid_bolus.head())



        stay_id            charttime  colloid_bolus
1650   30588857  2110-01-11 20:45:00            500
1651   30588857  2110-01-12 11:50:00            250
18606  36817845  2110-01-13 20:45:00            500
18607  36817845  2110-01-13 21:57:00            500
18608  36817845  2110-01-14 02:20:00            500
    stay_id            charttime  colloid_bolus
0  30588857  2110-01-11 20:45:00            500
1  30588857  2110-01-12 11:50:00            250
2  36817845  2110-01-13 20:45:00            500
3  36817845  2110-01-13 21:57:00            500
4  36817845  2110-01-14 02:20:00            500


In [24]:
from datetime import datetime, timedelta
import pandas as pd

df_bolus = pd.DataFrame(columns=['stay_id', 'Time', 'Bolus_value'])

def add_to_df(df, time, id, bolus_value):
    new_entry = pd.DataFrame({'stay_id': [id], 'Time': [time], 'Bolus_value': [bolus_value]})
    df = pd.concat([df, new_entry], ignore_index=True)
    return df

colloid_i = 0
crystalloid_i = 0
while colloid_i < len(df_colloid_bolus) and crystalloid_i < len(df_crystalloid_bolus):
    colloid_bolus_stay_id = df_colloid_bolus.iloc[colloid_i]['stay_id']
    colloid_bolus_charttime = df_colloid_bolus.iloc[colloid_i]['charttime']
    colloid_bolus_colloid_bolus = df_colloid_bolus.iloc[colloid_i]['colloid_bolus']
    
    crystalloid_bolus_stay_id = df_crystalloid_bolus.iloc[colloid_i]['stay_id']
    crystalloid_bolus_charttime = df_crystalloid_bolus.iloc[colloid_i]['charttime']
    crystalloid_bolus_colloid_bolus = df_crystalloid_bolus.iloc[colloid_i]['crystalloid_bolus']
    
    timestamp_compare_val = compare_timestamps(colloid_bolus_charttime, crystalloid_bolus_charttime) == -1
    if timestamp_compare_val == -1:
        colloid_i +=1
        df_bolus = add_to_df(df_bolus, colloid_bolus_charttime, colloid_bolus_stay_id, colloid_bolus_colloid_bolus)
    elif timestamp_compare_val == 1:
        crystalloid_i +=1
        df_bolus = add_to_df(df_bolus, crystalloid_bolus_charttime, crystalloid_bolus_stay_id, crystalloid_bolus_colloid_bolus)
    elif timestamp_compare_val == 0:
        crystalloid_i+=1
        colloid_i +=1
        
        df_bolus = add_to_df(df_bolus, crystalloid_bolus_charttime, crystalloid_bolus_stay_id, str(int(colloid_bolus_colloid_bolus) + int(crystalloid_bolus_colloid_bolus)))



    stay_id            charttime  colloid_bolus
0  30588857  2110-01-11 20:45:00            500
1  30588857  2110-01-12 11:50:00            250
2  36817845  2110-01-13 20:45:00            500
3  36817845  2110-01-13 21:57:00            500
4  36817845  2110-01-14 02:20:00            500


In [25]:
df_bolus.head()

,stay_id,Time,Bolus_value
0,30588857,2110-01-11 13:00:00,1000
1,30588857,2110-01-11 14:15:00,750
2,30588857,2110-01-11 15:53:00,1000
3,30588857,2110-01-11 17:16:00,1000
4,30588857,2110-01-12 01:50:00,1000


In [12]:

icuids = labs_df['stay_id'].unique()
treatments = ['norepinephrine', 'vasopressin', 'phenylephrine', 'dopamine', 'epinephrine']
vaso_treatments = load_treatments(data_path, icuids, treatments)

# select the vassopressin vaso_rate that are less than or equal to 0.2
vaso_treatments['vasopressin'] = vaso_treatments['vasopressin'][vaso_treatments['vasopressin']['vaso_rate'] <= 0.2]

# multiply the vaso_rate of vasopressin by 5
vaso_treatments['vasopressin']['vaso_rate'] = vaso_treatments['vasopressin']['vaso_rate']*5

# multiply the vaso_rate of phenylephrine by 0.45
vaso_treatments['phenylephrine']['vaso_rate'] = vaso_treatments['phenylephrine']['vaso_rate']*0.45

# multiply the vaso_rate of dopamine by 0.01
vaso_treatments['dopamine']['vaso_rate'] = vaso_treatments['dopamine']['vaso_rate']*0.01


finish reading norepinephrine.csv
finish reading vasopressin.csv
finish reading phenylephrine.csv
finish reading dopamine.csv
finish reading epinephrine.csv


In [ ]:
treatments = [

    'colloid_bolus', # -- fluid
    'crystalloid_bolus', #-- fluid
    'norepinephrine',  # --vaso
    'vasopressin',  # --vaso
    'phenylephrine',  # --vaso
    'dopamine', # --vaso
    'epinephrine',  # --vaso

    ]

#### Aggregate vitals and labs with static features (if any) and treatments (if any); 

Arguments
 
    dyn_df -- big union table of all vitals and labs from previous step
    impute_tb -- Table of imputation values
    static_df -- static info dataframe (default none)
    tr_df -- treatment dataframe dictionary from previous step (default {})
    traj_cut -- if cut trajectory to certain length (default 100,000, which is large enough to keep all)
    t_w -- feature binning time window length in hour (default 1)
    start_ts -- starting timestamp (if admit, using icu admission time; if treatment, use timestamp of first treatment; default treatment)
    binning -- how to bin treatment (irregular versus regular; default: irregular)
    norm_treatment -- if normalizing and aggregate treatment dosage, including vaso and fluid 
                           (default true; note not all vasos are implemented)
    fill_na -- if filling nans (if true use forward filling; if there are still nans, use impute table )



